In [13]:
from gurobipy import Model, quicksum, GRB

In [14]:
Plnt = {'plnt'}
DC = set(range(1,4))
Cust = DC.copy()

In [18]:
del_cost = {(j, k):10 for j in DC for k in Cust}

In [19]:
del_cost

{(1, 1): 10,
 (1, 2): 10,
 (1, 3): 10,
 (2, 1): 10,
 (2, 2): 10,
 (2, 3): 10,
 (3, 1): 10,
 (3, 2): 10,
 (3, 3): 10}

In [20]:
model = Model()

In [ ]:
x = {}

for j in 